In [ ]:
%run "autograd_lib.ipynb"

In [ ]:
# lint as python3
# Copyright 2019 Google LLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Autograd implementation of topology optimization for compliance minimization.
Exactly reproduces the result of "Efficient topology optimization in MATLAB
using 88 lines of code":
http://www.topopt.mek.dtu.dk/Apps-and-software/Efficient-topology-optimization-in-MATLAB
"""

# pylint: disable=missing-docstring
# pylint: disable=invalid-name
# pylint: disable=superfluous-parens

import autograd
import autograd.numpy as np

# A note on conventions:
# - forces and freedofs are stored flattened, but logically represent arrays of
#   shape (Y+1, X+1, 2)
# - mask is either a scalar (1) or an array of shape (X, Y).
# Yes, this is confusing. Sorry!


def run_topo_simp(x=None, args=None, loss_only=False, verbose=True):
  '''
  拓扑优化主函数，具体结构种类及优化参数以args字典的形式传递，下方specified_task()中展示了args字典的内容。
  一般生成args的流程是：首先针对具体结构生成一个problem类的实例，随后将problem传入utils中的specified_task()方法生成相应参数。
  '''
  # Root function that runs the full optimization routine
  if args is None:
    args = default_args()
  if x is None:
    x = np.ones((args['nely'], args['nelx'])) * args['volfrac']  # init mass

  if not loss_only:
    frames = [x.copy()]
  ke = get_stiffness_matrix(args['young'], args['poisson'])  # stiffness matrix

  losses = []
  sens = []
  for step in range(args['opt_steps']):
    c, dc, x = optimality_criteria_step(x, ke, args)
    losses.append(c)
    sens.append(dc)
    # if not loss_only and verbose and step % 5 == 0:
    #   print('step {}, loss {:.2e}'.format(step, c))

    if not loss_only:
      frames.append(x.copy())

  return losses[-1] if loss_only else (losses, frames, sens)

def specified_task(problem, opt_steps=50, filter_width=2):
  """Given a problem, return parameters for running a topology optimization."""
  fixdofs = np.flatnonzero(problem.normals.ravel())
  alldofs = np.arange(2 * (problem.width + 1) * (problem.height + 1))
  freedofs = np.sort(list(set(alldofs) - set(fixdofs)))

  params = {
      # material properties
      'young': 1,
      'young_min': 1e-9,
      'poisson': 0.3,
      'g': 0,
      # constraints
      'volfrac': problem.density,
      'xmin': 0.001,
      'xmax': 1.0,
      # input parameters
      'nelx': problem.width,
      'nely': problem.height,
      'mask': problem.mask,
      'freedofs': freedofs,
      'fixdofs': fixdofs,
      'forces': problem.forces.ravel(),
      'penal': 3.0,
      'rmin': 1.5,
      'filter_width': filter_width,
      'opt_steps': opt_steps
    }
  return params

def default_args():
  '''mbb梁案例作为默认参数'''
  # select the degrees of freedom
  nely = 25
  nelx = 80

  left_wall = list(range(0, 2*(nely+1), 2))
  right_corner = [2*(nelx+1)*(nely+1)-1]
  fixdofs = np.asarray(left_wall + right_corner)
  alldofs = np.arange(2*(nely+1)*(nelx+1))
  freedofs = np.asarray(list(set(alldofs) - set(fixdofs)))

  forces = np.zeros(2*(nely+1)*(nelx+1))
  forces[1] = -1.0

  return {'young': 1,     # material properties
          'young_min': 1e-9,
          'poisson': 0.3,
          'g': 0,  # force of gravity
          'volfrac': 0.4,  # constraints
          'nelx': nelx,     # input parameters
          'nely': nely,
          'freedofs': freedofs,
          'fixdofs': fixdofs,
          'forces': forces,
          'mask': 1,
          'penal': 3.0,
          'rmin': 1.5,
          'opt_steps': 50,
          'filter_width': 2,
          'step_size': 0.5,
          'name': 'truss'}

def optimality_criteria_step(x, ke, args, penal=3, e_min=1e-9, e_0=1):
  '''在每一迭代步中调用的优化准则法，输入当前步的单元密度，输出更新后的密度
  额外输出c和dc，其中c是当前步的目标函数值，用于记录目标函数下降历史，dc是灵敏度
  '''
  c, dc = autograd.value_and_grad(objective)(x, ke, args) # 输入变量关于目标函数的导数（灵敏度），通过autograd库中的方法计算
  dv = autograd.grad(mean_density)(x, args)
  x = optimality_criteria_combine(x, dc, dv, args)
  return c, dc, x

def optimality_criteria_combine(x, dc, dv, args, max_move=0.2, eta=0.5):
  '''优化准则法的实现'''
  """Fully differentiable version of the optimality criteria."""

  volfrac = args['volfrac']

  def pack(x, dc, dv):
    return np.concatenate([x.ravel(), dc.ravel(), dv.ravel()])

  def unpack(inputs):
    x_flat, dc_flat, dv_flat = np.split(inputs, [x.size, x.size + dc.size])
    return (x_flat.reshape(x.shape),
            dc_flat.reshape(dc.shape),
            dv_flat.reshape(dv.shape))

  def compute_xnew(inputs, lambda_):
    x, dc, dv = unpack(inputs)
    # avoid dividing by zero outside the design region
    dv = np.where(args['mask'] > 0, dv, 1)
    # square root is not defined for negative numbers, which can happen due to
    # small numerical errors in the computed gradients.
    xnew = x * np.maximum(-dc / (lambda_ * dv), 0) ** eta
    lower = np.maximum(0.0, x - max_move)
    upper = np.minimum(1.0, x + max_move)
    # note: autograd does not define gradients for np.clip
    return np.minimum(np.maximum(xnew, lower), upper)

  def f(inputs, lambda_):
    xnew = compute_xnew(inputs, lambda_)
    return volfrac - mean_density(xnew, args)

  # find_root allows us to differentiate through the while loop.
  inputs = pack(x, dc, dv)
  lambda_ = find_root(f, inputs, lower_bound=1e-9, upper_bound=1e9) # find_root函数在autograd_lib.ipynb中
  return compute_xnew(inputs, lambda_)

def objective(x, ke, args, cone_filter=True):
  '''求解目标函数的主函数。输入单元密度、结构边界条件和拓扑优化参数（包含在args字典中），输出目标函数值'''
  """Objective function (compliance) for topology optimization."""
  kwargs = dict(penal=args['penal'], e_min=args['young_min'], e_0=args['young'])
  x_phys = physical_density(x, args, apply_cone_filter=cone_filter)
  forces = calculate_forces(x_phys, args)
  u = displace(
      x_phys, ke, forces, args['freedofs'], args['fixdofs'], **kwargs)
  c = compliance(x_phys, u, ke, **kwargs)
  return c


def physical_density(x, args, apply_cone_filter=True):
  '''对单元密度施加过滤器cone_filter(cone_filter具体实现在autograd_lib.ipynb里，比较复杂)'''
  shape = (args['nely'], args['nelx'])
  assert x.shape == shape or x.ndim == 1
  x = x.reshape(shape)
  x = x * args['mask']
  if apply_cone_filter:
    x = cone_filter(x, args['filter_width'], args['mask'])
  return x

def calculate_forces(x_phys, args):
  '''
  将输入参数中力的矩阵提取出来，如果有自重这一参数则为力矩阵增加相应自重值。
  （本项目中没有考虑自重的情况）
  '''
  applied_force = args['forces']

  if not args.get('g'):
    return applied_force

  density = 0
  for pad_left in [0, 1]:
    for pad_up in [0, 1]:
      padding = [(pad_left, 1 - pad_left), (pad_up, 1 - pad_up)]
      density += (1/4) * np.pad(
          x_phys.T, padding, mode='constant', constant_values=0
      )
  gravitional_force = -args['g'] * density[..., np.newaxis] * np.array([0, 1])
  return applied_force + gravitional_force.ravel()

def displace(x_phys, ke, forces, freedofs, fixdofs, *,
             penal=3, e_min=1e-9, e_0=1):
  '''输入单元密度、单元刚度矩阵、边界条件，采用scipy库中的solve_coo()方法求解稀疏矩阵，获得各节点的位移'''
  # Displaces the load x using finite element techniques. The spsolve here
  # occupies the majority of this entire simulation's runtime.
  stiffness = young_modulus(x_phys, e_0, e_min, p=penal)
  k_entries, k_ylist, k_xlist = get_k(stiffness, ke)

  index_map, keep, indices = _get_dof_indices(
      freedofs, fixdofs, k_ylist, k_xlist
  )
  u_nonzero = solve_coo(k_entries[keep], indices, forces[freedofs],
                                     sym_pos=True)
  u_values = np.concatenate([u_nonzero, np.zeros(len(fixdofs))])

  return u_values[index_map]

def young_modulus(x, e_0, e_min, p=3):
  '''从单元密度到弹性模量的转化，加入极小值e_min防止数值计算方面的问题'''
  return e_min + x ** p * (e_0 - e_min)

def get_k(stiffness, ke):
  '''使用各单元密度和单元刚度矩阵组合成总刚度矩阵'''
  # Constructs a sparse stiffness matrix, k, for use in the displace function.
  nely, nelx = stiffness.shape

  # get position of the nodes of each element in the stiffness matrix
  ely, elx = np.meshgrid(range(nely), range(nelx))  # x, y coords
  ely, elx = ely.reshape(-1, 1), elx.reshape(-1, 1)

  n1 = (nely+1)*(elx+0) + (ely+0)
  n2 = (nely+1)*(elx+1) + (ely+0)
  n3 = (nely+1)*(elx+1) + (ely+1)
  n4 = (nely+1)*(elx+0) + (ely+1)
  edof = np.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
  edof = edof.T[0]

  x_list = np.repeat(edof, 8)  # flat list pointer of each node in an element
  y_list = np.tile(edof, 8).flatten()  # flat list pointer of each node in elem

  # make the stiffness matrix
  kd = stiffness.T.reshape(nelx*nely, 1, 1)
  value_list = (kd * np.tile(ke, kd.shape)).flatten()
  return value_list, y_list, x_list


def get_stiffness_matrix(young, poisson):
  '''平面应力问题，方形四节点单元的单元刚度矩阵，由弹性模量和泊松比确定'''
  # Element stiffness matrix
  e, nu = young, poisson
  k = np.array([1/2-nu/6, 1/8+nu/8, -1/4-nu/12, -1/8+3*nu/8,
                -1/4+nu/12, -1/8-nu/8, nu/6, 1/8-3*nu/8])
  return e/(1-nu**2)*np.array([[k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
                               [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
                               [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
                               [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
                               [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
                               [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
                               [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
                               [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]
                              ])

@ndarray_safe_lru_cache(1)
def _get_dof_indices(freedofs, fixdofs, k_xlist, k_ylist):
  '''求解稀疏矩阵时对向量索引进行处理的方法'''
  index_map = inverse_permutation(
      np.concatenate([freedofs, fixdofs]))
  keep = np.isin(k_xlist, freedofs) & np.isin(k_ylist, freedofs)
  i = index_map[k_ylist][keep]
  j = index_map[k_xlist][keep]
  return index_map, keep, np.stack([i, j])

def mean_density(x, args, cone_filter=True):
  '''求单元密度的平均值'''
  return (np.mean(physical_density(x, args, cone_filter))
          / np.mean(args['mask']))


def compliance(x_phys, u, ke, *, penal=3, e_min=1e-9, e_0=1):
  '''求解顺应度(单元应变能)'''
  # Calculates the compliance

  # index map
  nely, nelx = x_phys.shape
  ely, elx = np.meshgrid(range(nely), range(nelx))  # x, y coords

  # nodes
  n1 = (nely+1)*(elx+0) + (ely+0)
  n2 = (nely+1)*(elx+1) + (ely+0)
  n3 = (nely+1)*(elx+1) + (ely+1)
  n4 = (nely+1)*(elx+0) + (ely+1)
  all_ixs = np.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])

  # select from u matrix
  u_selected = u[all_ixs]

  # compute x^penal * U.T @ ke @ U in a vectorized way
  ke_u = np.einsum('ij,jkl->ikl', ke, u_selected)
  ce = np.einsum('ijk,ijk->jk', u_selected, ke_u)
  C = young_modulus(x_phys, e_0, e_min, p=penal) * ce.T
  return np.sum(C)

以上是SIMP方法的基本内容，下面是根据项目需要对原方法中间步骤的某些数据的提取。

In [ ]:
def run_topo_simp_decompose(x=None, args=None, verbose=True):
  '''求施加过滤器后的单元密度关于目标函数的导数（或称灵敏度），从而使本研究不受过滤器的限制'''
  # decompose dc into dc_phyx * dphyx_x
  if args is None:
    args = default_args()
  if x is None:
    x = np.ones((args['nely'], args['nelx'])) * args['volfrac']  # init mass

  ke = get_stiffness_matrix(args['young'], args['poisson'])  # stiffness matrix

  frames, sens, us = [], [], []
  for _ in range(args['opt_steps']):
    x_phys, u, dc_phyx, x = optimality_criteria_step_decompose(x, ke, args)
    sens.append(dc_phyx)
    frames.append(x_phys.copy())
    us.append(u)

  return frames, sens, us

def optimality_criteria_step_decompose(x, ke, args, penal=3, e_min=1e-9, e_0=1):
  '''求施加过滤器后的单元密度的灵敏度的子方法'''
  x_phys = physical_density(x, args, apply_cone_filter=True)
  kwargs = dict(penal=args['penal'], e_min=args['young_min'], e_0=args['young'])
  forces = calculate_forces(x_phys, args)
  u = displace(
      x_phys, ke, forces, args['freedofs'], args['fixdofs'], **kwargs)
  # index map
  nely, nelx = x_phys.shape
  ely, elx = np.meshgrid(range(nely), range(nelx))  # x, y coords
  # nodes
  n1 = (nely+1)*(elx+0) + (ely+0)
  n2 = (nely+1)*(elx+1) + (ely+0)
  n3 = (nely+1)*(elx+1) + (ely+1)
  n4 = (nely+1)*(elx+0) + (ely+1)
  all_ixs = np.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
  # select from u matrix
  u_selected = u[all_ixs]
  
  # compute x^penal * U.T @ ke @ U in a vectorized way
  ke_u = np.einsum('ij,jkl->ikl', ke, u_selected)
  ce = np.einsum('ijk,ijk->jk', u_selected, ke_u)
  dc_phyx = -penal * young_modulus(x_phys, e_0, e_min, p=penal-1) * ce.T
  dc = cone_filter(dc_phyx, args['filter_width'], args['mask'], transpose=True) # 用transpose后的cone_filter对dc_phyx作处理即得到dc。
  dv = autograd.grad(mean_density)(x, args)
  x = optimality_criteria_combine(x, dc, dv, args)

  return x_phys, u, dc_phyx, x 

def fem_solver(x_phys, args, penal=3, e_min=1e-9, e_0=1):
  '''给定单元密度和结构描述，求其节点位移及单元灵敏度。'''
  '''receive density and problem args, return displacements & sensitivities'''
  ke = get_stiffness_matrix(args['young'], args['poisson'])
  kwargs = dict(penal=args['penal'], e_min=args['young_min'], e_0=args['young'])
  forces = calculate_forces(x_phys, args)
  u = displace(
      x_phys, ke, forces, args['freedofs'], args['fixdofs'], **kwargs)
  # index map
  nely, nelx = x_phys.shape
  ely, elx = np.meshgrid(range(nely), range(nelx))  # x, y coords
  # nodes
  n1 = (nely+1)*(elx+0) + (ely+0)
  n2 = (nely+1)*(elx+1) + (ely+0)
  n3 = (nely+1)*(elx+1) + (ely+1)
  n4 = (nely+1)*(elx+0) + (ely+1)
  all_ixs = np.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
  # select from u matrix
  u_selected = u[all_ixs]

  # compute x^penal * U.T @ ke @ U in a vectorized way
  ke_u = np.einsum('ij,jkl->ikl', ke, u_selected)
  ce = np.einsum('ijk,ijk->jk', u_selected, ke_u)
  dc_phyx = -penal * young_modulus(x_phys, e_0, e_min, p=penal-1) * ce.T

  return u, dc_phyx

def compliance_solver(x, args, penal=3, e_min=1e-9, e_0=1):
  '''给定单元密度和结构描述，求其顺应度（目标函数）'''
  x_phys = physical_density(x, args)
  ke = get_stiffness_matrix(args['young'], args['poisson'])
  kwargs = dict(penal=args['penal'], e_min=args['young_min'], e_0=args['young'])
  forces = calculate_forces(x_phys, args)
  u = displace(
      x_phys, ke, forces, args['freedofs'], args['fixdofs'], **kwargs)
  # index map
  nely, nelx = x_phys.shape
  ely, elx = np.meshgrid(range(nely), range(nelx))  # x, y coords
  # nodes
  n1 = (nely+1)*(elx+0) + (ely+0)
  n2 = (nely+1)*(elx+1) + (ely+0)
  n3 = (nely+1)*(elx+1) + (ely+1)
  n4 = (nely+1)*(elx+0) + (ely+1)
  all_ixs = np.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
  # select from u matrix
  u_selected = u[all_ixs]

  # compute x^penal * U.T @ ke @ U in a vectorized way
  ke_u = np.einsum('ij,jkl->ikl', ke, u_selected)
  ce = np.einsum('ijk,ijk->jk', u_selected, ke_u)
  C = young_modulus(x_phys, e_0, e_min, p=penal) * ce.T

  return np.sum(C)